In [1]:
import numpy as np
from rubin_sim.satellite_constellations import Constellation, starlink_tles_v1, starlink_tles_v2
import pandas as pd
from rubin_sim.utils import point_to_line_distance

In [2]:
# need to add a new method on the constallation to do the checking the way we want
class SConstellation(Constellation):
    def check_positions(self, positions_ra, positions_dec, mjds, visit_times, mask_width=1./60.):
        """Check if RA,dec,mjd spot is hit by a satellite streak
        
        Parameters
        ----------
        positions_ra : np.array
            The RA of each solar system detection (degrees).
        positions_dec : np.array
            The Dec of each solar system detection (degrees).
        mjds : np.array
            The MJD of each detection
        visit_times : np.array
            The total visit times for each detection. Typically exposure time plus 
            any additional read time or shutter motion time. (seconds)
        mask_width : float
            The width of the expected streak (degrees)
        
        Returns
        -------
        index values for positions that were hit by a streak
        """
        # convert all input to radians
        positions_ra = np.radians(positions_ra)
        positions_dec = np.radians(positions_dec)
        mask_width = np.radians(mask_width)
        
        visit_times = visit_times / 3600.0 / 24.0  # convert seconds to days
        
        input_id_indx_oned = np.arange(positions_ra.size, dtype=int)
        
        sat_ra_1, sat_dec_1, sat_alt_1, sat_illum_1 = self.paths_array(mjds)
        mjd_end = mjds + visit_times
        sat_ra_2, sat_dec_2, sat_alt_2, sat_illum_2 = self.paths_array(mjd_end)
    
        
        # broadcast the object positions to be the same shape as the satellite arrays.
        pointing_ras_rad = np.broadcast_to(positions_ra, sat_ra_1.shape)
        pointing_decs_rad = np.broadcast_to(positions_dec, sat_ra_1.shape)
        input_id_indx = np.broadcast_to(input_id_indx_oned, sat_ra_1.shape)
        
        above_illum_indx = np.where(
            ((sat_alt_1 > self.alt_limit_rad) | (sat_alt_2 > self.alt_limit_rad))
            & ((sat_illum_1 == True) | (sat_illum_2 == True))
        )

        # XXXX--this is assuming satellites travel on straight lines on the sphere
        # I'm not sure that's a super great assumption, but maybe works out
        # statistically where we get some hits right and some wrong
        
        # point_to_line_distance can take arrays, but they all need to be the same shape,
        # thus why we broadcast pointing ra and dec above.
        # This might be better done with a KD tree. Especially if cranking up to 
        # very large satellite constellations.
        distances = point_to_line_distance(
            sat_ra_1[above_illum_indx],
            sat_dec_1[above_illum_indx],
            sat_ra_2[above_illum_indx],
            sat_dec_2[above_illum_indx],
            pointing_ras_rad[above_illum_indx],
            pointing_decs_rad[above_illum_indx],
        )

        close = np.where(distances < mask_width)[0]
        
        # Could set a "close pass" value, maybe one degree, use that instead of the
        # mask width above, then for each indx in close, do a higher time-resolution
        # calculation of the satllite path and check if it gets within mask_width.
        # there is an example in the original check_pointings method on the base class
        
        # Note, could have repeat values in result as an object can be hit by more than one streak
        return input_id_indx[above_illum_indx][close]

In [3]:
ss_observations_file = 'franken_v2.99_10yrs__vatiras_granvik_10k_obs.txt'
ss_objects = pd.read_csv(ss_observations_file, delim_whitespace=True, comment="#")

In [4]:
ss_objects

,obj_id,time,ra,dec,dradt,ddecdt,phase,solarelon,helio_dist,geo_dist,...,night,observationStartMJD,rotSkyPos,seeingFwhmEff,seeingFwhmGeom,solarElong,visitExposureTime,dmag_color,dmag_trail,dmag_detect
0,11,62967.419811,338.404819,-0.089874,0.499149,0.651762,90.776102,41.584411,0.665140,0.740439,...,2749,62967.419811,48.924133,1.195006,1.034295,41.448783,30.0,-0.399899,0.162093,0.187372
1,11,62968.419071,338.930048,0.564598,0.533552,0.658340,90.422628,41.887308,0.669243,0.741201,...,2750,62968.419071,80.032867,1.353303,1.164415,41.664981,30.0,-0.264722,0.144988,0.161555
2,26,62253.422266,355.258715,0.577490,0.589892,0.468475,106.782836,41.210646,0.692690,0.557195,...,2035,62253.422266,169.591668,1.754406,1.494122,40.865009,30.0,-0.264722,0.087304,0.083603
3,26,63314.414096,322.743882,-13.926479,0.802013,0.411934,79.787689,42.301025,0.681688,0.858070,...,3096,63314.414096,50.825246,1.333734,1.148329,43.336650,15.0,-0.399899,0.059924,0.052230
4,26,63318.415342,326.188198,-12.216291,0.860172,0.436694,77.957553,42.545312,0.689976,0.879132,...,3100,63318.415342,189.915647,1.786168,1.520230,42.544470,15.0,-0.264722,0.042753,0.034688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6805,9982,62170.038460,1.552118,-30.109421,0.555723,-0.277885,106.338700,43.166854,0.702712,0.521183,...,1952,62170.038460,291.162761,1.258755,1.086696,44.047758,15.0,-0.457036,0.036052,0.028332
6806,9982,62854.353028,217.589546,-35.600021,1.076303,-0.150642,84.303241,44.028253,0.687269,0.775638,...,2636,62854.353028,101.233635,2.193223,1.854829,44.009572,15.0,-0.457036,0.037536,0.029715
6807,9982,62854.356130,217.593651,-35.600488,1.076023,-0.150529,84.301905,44.027698,0.687264,0.775659,...,2636,62854.356130,100.707226,2.162433,1.829520,44.012270,15.0,-0.457036,0.038376,0.030503
6808,9990,60302.356291,221.063588,-32.174642,0.820819,-0.296923,95.277325,45.565456,0.705353,0.623704,...,84,60302.356291,31.944435,1.251153,1.080448,46.697326,15.0,-0.457036,0.062624,0.055144


In [5]:
tles = starlink_tles_v2()
print('N satellites = ', np.size(tles))
constellation = SConstellation(tles)

N satellites =  29988


In [6]:
visit_times = ss_objects['visitExposureTime']+2


In [7]:
# this step is the grind. Generates a few arrays that are n_detections X n_satellites, so
# potential to gobble up a few GB of memory. Looks like I got up to 10-15 GB on a 6.8k x 30k run.
indx = constellation.check_positions(ss_objects['ra'].values, ss_objects['dec'].values,
                                     ss_objects['observationStartMJD'].values, visit_times.values)
# Note that if you want to check something like a 300k constellation, one can break it up into 
# 10 30k constellations and run in parallel (or whatever) and then just concatenate the indx arrays.

In [8]:
print(indx.size)
indx

27


array([4592, 3889, 5236, 3498,  549, 4475, 3067, 3731, 6132, 4978, 3549,
       1412, 2726, 1692, 3678,  893, 5155, 5097, 6769,  807, 5430, 3655,
       3650, 3698, 2022, 5633, 3560])

In [9]:
# so, we really didn't lose that many observations at all, 27 out of 6810
# let's drop those out of the dataframe
final_data = ss_objects.drop(index=indx)

In [10]:
final_data.shape

(6783, 27)

In [11]:
# And now we can put that back as a new txt file with the streaked objects removed
final_data.to_csv('franken_v2.99_10yrs__vatiras_granvik_10k_obs_streaked.txt', index=False, sep=' ')

In [12]:
27/6783

0.003980539584254755

In [ ]:
#Wow, I think this exactly matches what we had in the paper as well. 0.4% loss of detections. 